In [33]:
import matplotlib.pyplot as plt
import numpy as np
import cPickle

import analysis
reload(analysis)

<module 'analysis' from 'analysis.py'>

In [52]:
fname = '/home/jesse/Downloads/comparison_128_2.pkl'

W_priors = ['L2', 'L4', 'RANDOM']
ica_models = [2, 4, 6, 8, 'RANDOM', 'COULOMB', 'COHERENCE'] + ['SC']
lambdas = np.logspace(-2, 2, num=9)
colors = ['black', 'black', 'black', 'black', 'red', 'green', 'blue', 'purple']
fmts = ['-', '--', ':', '-.', '-', '-', '-', '-']

In [3]:
with open(fname, 'r') as f:
    data = cPickle.load(f)

In [4]:
A_dict, W_dict, W_fits, min_ks, results = data
n_mixtures, n_sources = A_dict[A_dict.keys()[0]][0].shape
n_iter = W_fits.shape[3]
results = np.nan * np.ones((len(W_priors), len(ica_models), lambdas.size, n_iter, 2))

In [34]:
for ii, p in enumerate(W_priors):
    for jj, m in enumerate(ica_models):
        A = A_dict[p][jj]
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                W = W_fits[ii, jj, kk, ll]
                results[ii, jj, kk, ll] = analysis.recovery_statistics_AW(A, W)

In [53]:
for ii, p in enumerate(W_priors):
    fd = plt.figure()
    ad = fd.add_subplot(1, 1, 1)
    fe = plt.figure()
    ae = fe.add_subplot(1, 1, 1)
    for jj, m in enumerate(ica_models):
        dist = results[ii, jj, :, :, 0].mean(axis=1)
        print results[ii, jj, :, :, 1]
        delta = results[ii, jj, :, :, 1].mean(axis=1)
        ad.semilogx(lambdas, dist, fmts[jj], label=str(m), c=colors[jj])
        ae.semilogx(lambdas, delta, fmts[jj], label=str(m), c=colors[jj])
    ad.set_title(p)
    ad.set_xlabel('Lambda')
    ad.set_ylabel('Hellinger Distance')
    ad.legend()
    ae.set_title(p)
    ae.set_xlabel('Lambda')
    ae.set_ylabel('Permutation Delta')
    ae.legend()
plt.show()
plt.close()

[[  58.  168.  184.  170.  172.  176.  170.  180.  180.  170.]
 [  32.  178.  162.  176.  164.  162.  168.  186.  166.  196.]
 [  24.  164.  166.  176.  164.  174.  178.  174.  170.  188.]
 [  22.  172.  180.  184.  170.  166.  186.  184.  172.  188.]
 [  22.  164.  190.  184.  170.  174.  168.  200.  162.  166.]
 [  30.  168.  194.  166.  186.  174.  174.  192.  166.  182.]
 [  52.  162.  174.  180.  174.  178.  170.  174.  164.  180.]
 [  84.  166.  182.  150.  192.  184.  170.  192.  182.  184.]
 [ 122.  214.  202.  190.  228.  220.  204.  216.  192.  204.]]
[[ 176.   66.  176.  166.  180.  172.  166.  168.  162.  184.]
 [ 182.    6.  182.  184.  174.  190.  186.  182.  178.  160.]
 [ 172.    0.  176.  192.  182.  186.  182.  176.  174.  170.]
 [ 170.    0.  162.  184.  182.  196.  182.  166.  176.  164.]
 [ 160.    2.  152.  180.  190.  172.  170.  162.  172.  176.]
 [ 166.   62.  188.  188.  190.  172.  180.  170.  184.  178.]
 [ 182.   80.  188.  182.  200.  188.  184.  182.  184